#### Chapter5. 오차역전파법

##### 5.4 단순한 계층 구현하기

5.4.1 곱셈 계층

In [42]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
    
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y

        return out
    
    def backward(self, dout):
        dx = dout * self.y  # x와 y를 바꾼다.
        dy = dout * self.x

        return dx, dy

In [43]:
apple = 100
apple_num = 2
tax = 1.1

# 계층들
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# 순전파 구현
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(price)

220.00000000000003


In [44]:
# 역전파
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax)

2.2 110.00000000000001 200


5.4.2 덧셈 계층

In [45]:
class AddLayer:
    # 덧셈 계층에서는 초기화가 필요 없음
    def __init__(self):
        pass

    def forward(self, x, y):
        out = x + y
        return out
    
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy

In [46]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# 계층들
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax)

# 역전파
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(price)
print(dapple_num, dapple, dorange, dorange_num, dtax)

715.0000000000001
110.00000000000001 2.2 3.3000000000000003 165.0 650


##### 활성화 함수 계층 구현하기

5.5.1 ReLU 계층

In [47]:
class Relu:
    def __init__(self):
        self.mask = None
    
    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout

        return dx

In [48]:
import numpy as np
x = np.array([[1.0, -0.5], [-2.0, 3.0]])
print(x)

[[ 1.  -0.5]
 [-2.   3. ]]


In [49]:
mask = (x <= 0)
print(mask)

[[False  True]
 [ True False]]


5.5.2 Sigmoid 계층

In [50]:
class Sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
    
    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out

        return dx

##### 5.6 Affine / Softmax 계층 구현하기

5.6.1 Affine 계층

- affine transformation(어파인 변환) : 행렬의 곱

In [51]:
X = np.random.rand(2)
W = np.random.rand(2, 3)
B = np.random.rand(3)

In [52]:
print(X.shape)
print(W.shape)
print(B.shape)

(2,)
(2, 3)
(3,)


In [53]:
# 뉴런의 가중치 합
Y = np.dot(X, W) + B

5.6.2 배치용 Affine 계층

In [54]:
# 순전파 편향 덧셈 : N = 2(데이터 2개)
X_dot_W = np.array([[0, 0, 0], [10, 10, 10]])
B = np.array([1, 2, 3])

In [55]:
X_dot_W

array([[ 0,  0,  0],
       [10, 10, 10]])

In [56]:
X_dot_W + B

array([[ 1,  2,  3],
       [11, 12, 13]])

In [57]:
# 역전파 => 데이터의 역전파 값이 편향의 원소에 모여야 함
dY = np.array([[1, 2, 3], [4, 5, 6]])
dY

array([[1, 2, 3],
       [4, 5, 6]])

In [58]:
dB = np.sum(dY, axis=0)
dB

array([5, 7, 9])

In [59]:
# Affine 구현
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = B
        self.x = None
        self.dW = None
        self.db = None

    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b

        return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)

        return dx

5.6.3 Softmax-with-Loss 계층

📌신경망 학습의 목적 : 신경망의 출력이 정답 레이블과 가까워지도록 가중치 매개변수의 값 조정

In [60]:
from common.functions import softmax, cross_entropy_error

In [61]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None

    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size

        return dx

##### 5.7 오차역전파법 구현하기

5.7.1 신경망 학습의 전체 그림

1. 미니배치 : 훈련 데이터 중 일부를 무작위로 가져옴
2. 기울기 산출 : 각 가중치 매개변수의 기울기 구함 (`오차역전파법` 사용)
3. 매개변수 갱신 : 가중치 매개변수를 기울기 방향으로 조금씩 갱신
4. 반복

5.7.2 오차역전파법을 적용한 신경망 구현하기

In [62]:
import sys, os
sys.path.append(os.pardir)
import numpy as np

In [63]:
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict

In [64]:
class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

        # 계층 생성
        self.layers = OrderedDict()     # 데이터의 순서 고려(딕셔너리에 추가한 순서 기)
        self.layers['Affine'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])

        self.lastLayer = SoftmaxWithLoss()

    # 예측 수행
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        
        return X
    
    # 손실함수
    # x : input data, t : target label
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    # 정확도
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1: t = np.argmax(t, axis=1)

        accuracy = np.sum(y == t) / float(x.shape[0])
        
        return accuracy
    
    # 가중치 매개변수의 기울기를 수치 미분 방식으로 구함
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)

        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        return grads
    
    # 가중치 매개변수의 기울기를 오차역전파법으로 구함
    def gradient(self, x, t):
        # 순전파
        self.loss(x, t)

        # 역전파
        dout = 1
        dout = self.lastLayer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # save result
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db

        return grads